# Asthma-Like Illness Emergency Department Presentations (monthly) | Processing

The main tasks completed to clean and preprocess this dataset were:

**Data Manipulation**
1. Rename columns.
2. Move 'date' column to the first position.
3. Remove 'LHD' from LHD name values.
4. Remove 'All' data (Representing a state-wide average).
5. Remove columns holding Confidence Interval data.
6. Remove rows holding 'Persons' data in the sex column (Representing a genderless rate per 100,000).

## Set Up

Ensure that the required libraries are available by running the below code in the terminal before execution:
- pip install pandas


Execute the following in the jupyter notebook before execution to ensure that the required libraries are imported:

In [23]:
import pandas as pd

## Load Dataset

In [24]:
# File path.
file_path = 'data-raw.csv'

# Read the file.
df = pd.read_csv(file_path)

## Data Manipulation

Rename columns to match Air Quality data set.

In [25]:
# Rename columns.
df = df.rename(columns={
    'LHD': 'lhd',
    'Period': 'date'
})

# Set column names to lower case.
df.columns = df.columns.str.lower()

Move 'date' column to the first position.

In [26]:
# Move 'date' column to the first position.
date_col = df.pop('date')
df.insert(0, 'date', date_col)

Remove ' LHD' for Local Health District values.

In [27]:
# Remove ' LHD' from the 'lhd' column.
df['lhd'] = df['lhd'].str.replace(' LHD', '')

Remove rows representing state-wide aggregated data.

In [28]:
# Remove rows with 'All' in the 'lhd' column.
df = df[~df['lhd'].str.contains('All')]

Remove columns holding Confidence Interval data.

In [29]:
# Drop columns with '% ci' in the header
df = df.loc[:, ~df.columns.str.contains('% ci')]

Remove rows holding 'Persons' data in the sex column.

In [30]:
# Drop rows with 'Persons' in the 'sex' column.
df = df[~df['sex'].str.contains('Persons')]

## Output Processed Dataset

In [31]:
# File path.
file_path_output = 'data-processed.csv'

# Save the file.
df.to_csv(file_path_output, index=False)

## View Dataset

In [32]:
df

,date,sex,lhd,"rate per 100,000 population"
0,2014-07,Males,Sydney,22.6
1,2014-08,Males,Sydney,28.9
2,2014-09,Males,Sydney,15.7
3,2014-10,Males,Sydney,19.2
4,2014-11,Males,Sydney,19.7
...,...,...,...,...
3127,2023-02,Females,Western NSW,41.4
3128,2023-03,Females,Western NSW,43.3
3129,2023-04,Females,Western NSW,43.9
3130,2023-05,Females,Western NSW,57.7


## Alternative Approach
Composite primary key becomes only 'lhd' and 'date'.

Tracks 'Male rate per 100,000 population' and 'Female rate per 100,000 population' on the same row.

### Reconfigure Table

In [33]:
# Pivot the dataframe to have 'sex' as columns
df_alt = df.pivot_table(index=['lhd', 'date'], columns='sex', values='rate per 100,000 population').reset_index()

# Rename the columns to match the desired format
df_alt.columns.name = None
df_alt = df_alt.rename(columns={'Males': 'Male rate per 100,000 population', 'Females': 'Female rate per 100,000 population'})

df_alt

,lhd,date,"Female rate per 100,000 population","Male rate per 100,000 population"
0,Central Coast,2014-07,29.8,21.8
1,Central Coast,2014-08,34.3,32.0
2,Central Coast,2014-09,28.4,19.9
3,Central Coast,2014-10,28.0,19.5
4,Central Coast,2014-11,28.0,21.7
...,...,...,...,...
1507,Western Sydney,2023-02,19.3,23.6
1508,Western Sydney,2023-03,16.2,14.0
1509,Western Sydney,2023-04,22.1,18.0
1510,Western Sydney,2023-05,22.9,21.9


### Output Alternative Processed Dataset

In [34]:
# File path.
file_path_output_alt = 'data-processed-alt.csv'

# Save the file.
df_alt.to_csv(file_path_output_alt, index=False)